In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise import KNNBasic

ModuleNotFoundError: No module named 'surprise'

这下面是是基于模型推荐算法，后面有基于用户协同过滤和基于物品协同过滤，相同的输入不同的输出

In [2]:
ratings = pd.read_csv('bigdata dataset/ratings_small.csv')

movie_md = pd.read_csv('bigdata dataset/movies_metadata.csv')

ratings.head()

/var/folders/4k/lgt68y3s42gdrfy_qbk6_hwc0000gn/T/ipykernel_54960/2189024615.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_md = pd.read_csv('bigdata dataset/movies_metadata.csv')


userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

In [3]:
movie_md = movie_md[movie_md['vote_count']>55][['id','title']]
movie_ids = [int(x) for x in movie_md['id'].values]

ratings = ratings[ratings['movieId'].isin(movie_ids)]
ratings.reset_index(inplace=True, drop=True)

ratings.head()

userId  movieId  rating   timestamp
0       1     1371     2.5  1260759135
1       1     2105     4.0  1260759139
2       1     2294     2.0  1260759108
3       2       17     5.0   835355681
4       2       62     3.0   835355749

In [4]:
ratings.shape

(29965, 4)

In [5]:
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0,5), skip_lines=1)

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader=reader)

trainset = data.build_full_trainset()

In [6]:
svd = SVD()
svd.fit(trainset)

In [8]:
svd.predict(3,2959,5.0)

Prediction(uid=3, iid=2959, r_ui=5.0, est=3.911876362440694, details={'was_impossible': False})

In [14]:
def get_recommendations(data, movie_md, user_id, top_n, algo):
    
    recommendations = []
    user_movie_interactions_matrix = data.pivot(index='userId', columns='movieId', values='rating')
    non_interacted_movies = user_movie_interactions_matrix.loc[user_id][user_movie_interactions_matrix.loc[user_id].isnull()].index.tolist()
    
    for item_id in non_interacted_movies:
        
        est = algo.predict(user_id, item_id).est
        
        movie_name = movie_md[movie_md['id']==str(item_id)]['title'].values[0]
        recommendations.append((movie_name, est))

    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:top_n] 

In [15]:
get_recommendations(data=ratings,movie_md=movie_md, user_id=654, top_n=10, algo=svd)

[('Amélie', 4.899191333579904),
 ('Madagascar', 4.894314593081367),
 ('The Thomas Crown Affair', 4.893802087505915),
 ('The Thomas Crown Affair', 4.872858320798504),
 ('While You Were Sleeping', 4.870776935367764),
 ('Birdman of Alcatraz', 4.826521810968868),
 ('Nell', 4.82434657307676),
 ('Laura', 4.803307561497861),
 ('Rumble Fish', 4.800709870455692),
 ('My Darling Clementine', 4.781284729930374)]

下面是👇User-base

In [16]:
sim_options = {'name': 'cosine',
               'user_based': True}

sim_user = KNNBasic(sim_options= sim_options, verbose=False, random_state = 33)

sim_user.fit(trainset=data.build_full_trainset())

In [17]:
sim_user.predict(2,17,5)

Prediction(uid=2, iid=17, r_ui=5, est=4.166335018545322, details={'actual_k': 40, 'was_impossible': False})

In [18]:
get_recommendations(ratings, movie_md, 671, 10, sim_user)

[('The Wizard', 5),
 ('Rio Bravo', 5),
 ('The Celebration', 5),
 ('Spider-Man 3', 5),
 ('A Streetcar Named Desire', 5),
 ('Gentlemen Prefer Blondes', 5),
 ('The Evil Dead', 5),
 ('JFK', 5),
 ('Strangers on a Train', 5),
 ("Singin' in the Rain", 5)]

下面是👇Item-base

In [19]:
sim_options = {'name': 'cosine',
               'user_based': False}

sim_item = KNNBasic(sim_options= sim_options, verbose=False, random_state = 33)

sim_item.fit(trainset=data.build_full_trainset())

In [21]:
sim_item.predict(2,17,5)

Prediction(uid=2, iid=17, r_ui=5, est=3.650476877827318, details={'actual_k': 40, 'was_impossible': False})

In [20]:
get_recommendations(ratings, movie_md, 671, 10, sim_item)

[('Hard Candy', 5),
 ('Visitor Q', 5),
 ('The Protector', 4.666666666666667),
 ('Shaun of the Dead', 4.571428571428571),
 ('The Silence of the Lambs', 4.503228000162119),
 ("Singin' in the Rain", 4.5),
 ("Hearts of Darkness: A Filmmaker's Apocalypse", 4.5),
 ('Sense and Sensibility', 4.5),
 ("The Hitchhiker's Guide to the Galaxy", 4.5),
 ('Fantasia', 4.428571428571429)]